# ランドマーク選択 (Steps A–C)

## Step A: k近傍グラフの構築

データ点 \(x_i \in \mathbb{R}^m\) から **k近傍グラフ (kNN Graph)** を作成する。  

各点 \(x_i\) に対して局所的に適応したカーネル関数（UMAPと同じ）を計算する：

\[
p_{i|j} = \exp \left( -\frac{d(x_i, x_j) - \rho_i}{\sigma_i} \right)
\]

- \(d(x_i, x_j)\): ユークリッド距離  
- \(\rho_i\): 最近傍距離  
- \(\sigma_i\): バイナリサーチで決めるスケーリング値（kを満たすように調整）  

---

## Step B: 有限マルコフ連鎖 (Finite Markov Chain, FMC) の構築

遷移確率は次のように定義する：

\[
T_{ij} = \frac{p_{i|j}}{\sum_k p_{i|k}}, \quad j \in NH(i)
\]

ここで \(NH(i)\) は点 \(x_i\) の近傍集合。  
この遷移確率行列を用いてランダムウォークをシミュレーションする。  

---

## Step C: ランドマーク選択

1. 各ノード \(i\) から \(n\) 回、長さ \(\mu\) のランダムウォークを実行する。  
2. ランダムウォークのエンドポイントをカウントする。  
3. 最も訪問回数が多いノードを「ランドマーク」として選択する。  
4. 指定数 \(|H_i|\) のランドマークを次の階層に選ぶ。  


In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

import pynndescent

def compute_probabilities(X, k=15, knn_algorithm="sklearn"):
    """
    kNN探索と確率行列を計算
    knn_algorithm: "sklearn" または "NNDescent"
    """
    n = X.shape[0]

    if knn_algorithm == "sklearn":
        nbrs = NearestNeighbors(n_neighbors=k+1, algorithm="auto").fit(X)
        dists, indices = nbrs.kneighbors(X)

    elif knn_algorithm == "NNDescent":
        nnd = pynndescent.NNDescent(X, n_neighbors=k+1, metric="euclidean")
        indices, dists = nnd.neighbor_graph

        # pynndescent は (indices, distances) の形で返す
        indices = indices[:, :k+1]
        dists = dists[:, :k+1]

    else:
        raise ValueError(f"Unknown knn_algorithm: {knn_algorithm}")

    # 自分自身を除外
    dists, indices = dists[:, 1:], indices[:, 1:]

    P = np.zeros((n, k))
    for i in range(n):
        rho_i = dists[i, 0]  # 最近傍距離

        def perplexity_error(sigma):
            p = np.exp(-(dists[i] - rho_i) / sigma)
            return np.sum(p) - k

        sigma = binary_search(perplexity_error, 1e-5, 100)
        P[i] = np.exp(-(dists[i] - rho_i) / sigma)
        P[i] /= np.sum(P[i])

    return indices, P

# 
def binary_search(f, low, high, tol=1e-5):
    for _ in range(50):
        mid = (low + high) / 2
        val = f(mid)
        if abs(val) < tol:
            return mid
        if val > 0:
            high = mid
        else:
            low = mid
    return mid

def select_landmarks(X, num_landmarks=50, k=100, n_walks=10, walk_length=10, knn_algorithm="sklearn"):
    indices, P = compute_probabilities(X, k=k, knn_algorithm=knn_algorithm)
    n = X.shape[0]
    visits = np.zeros(n)

    for start in range(n):
        for _ in range(n_walks):
            cur = start
            for _ in range(walk_length):
                probs = P[cur]
                neighbors = indices[cur]
                cur = np.random.choice(neighbors, p=probs)
            visits[cur] += 1

    # 最も訪問数が多いノードをランドマークにする
    landmark_ids = np.argsort(-visits)[:num_landmarks]
    return landmark_ids, visits


In [ ]:
import time
X = np.random.rand(10000, 50)  # 例としてランダムデータ
y = np.random.randint(0, 10, size=(10000,))

In [ ]:


start_time = time.time()
landmark_ids, visits = select_landmarks(X, num_landmarks=2000, k=100, n_walks=10, walk_length=10, knn_algorithm="NNDescent")

print(f"ランドマーク選択にかかった時間: {time.time() - start_time}秒")
print(f"選ばれたランドマークのインデックス: {len(landmark_ids)}")

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


ランドマーク選択にかかった時間: 19.01636505126953秒
選ばれたランドマークのインデックス: 2000


In [23]:
import humap

# test humap
# hUmap1 = humap.UMAP(knn_algorithm="NNDescent", reproducible=True)
# fitting
hUmap = humap.HUMAP(np.array([0.2, 0.2]), verbose=True)
hUmap.fit(X, y)


L0 - 10000 data samples.
L0 - Fitting: done in 4.514105 seconds.

L1 - 2000 data samples.
L1 - Selecting Landmarks: done in 0.015961 seconds.
L1 - Constructing Neighborhood: done in 0.056878 seconds.
L1 - Sparse Similarity: done in 0.022018 seconds.
L1 - Fitting: done in 0.038594 seconds.
L1 - Associating data points to landmarks: done in 0.003228 seconds.
L1 - Construction: done in 0.202776

L2 - 400 data samples.
L2 - Selecting Landmarks: done in 0.003460 seconds.
L2 - Constructing Neighborhood: done in 0.011765 seconds.
L2 - Sparse Similarity: done in 0.001970 seconds.
L2 - Fitting: done in 0.005147 seconds.
L2 - Associating data points to landmarks: done in 0.000407 seconds.
L2 - Construction: done in 0.039394

Hierarchical Representation: done in 4.768219 seconds.


In [24]:
# humapのランドマークを取得 l1
l1_landmarks = hUmap.original_indices(1)  # レベル1のランドマークの元データインデックス
print(f"レベル1のランドマーク数: {len(l1_landmarks)}")


レベル1のランドマーク数: 2000


In [25]:
# 実装とhumapのインデックスの一致具合を確認
print(f"ランドマークのインデックスの一致数: {len(set(landmark_ids) & set(l1_landmarks))}"   )

ランドマークのインデックスの一致数: 1535


# Dissimilairy 

In [26]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def compute_rnh_dissimilarity(X_prev, landmarks, k=15, beta=0.0, n_walks=10, walk_length=10):
    """
    X_prev: 下位階層のデータ点 (Hi-1)
    landmarks: 上位階層のランドマークインデックス (Hi)
    k: kNN の数
    beta: local neighborhood の重み (0~1)
    n_walks, walk_length: ランダムウォークの回数と長さ
    """
    n_prev = X_prev.shape[0]
    n_landmarks = len(landmarks)

    # 下位レベルの kNN グラフを作成
    nbrs = NearestNeighbors(n_neighbors=k+1).fit(X_prev)
    dists, indices = nbrs.kneighbors(X_prev)
    dists, indices = dists[:, 1:], indices[:, 1:]  # 自分自身は除外

    # Representation Neighborhood (RNH) 行列の初期化
    RNH = np.zeros((n_landmarks, n_prev))

    # Global neighborhood: ランダムウォークで RNH を構築
    for start in range(n_prev):
        for _ in range(n_walks):
            cur = start
            for _ in range(walk_length):
                cur_neighbors = indices[cur]
                # 簡易確率として一様に選択
                cur = np.random.choice(cur_neighbors)
            # ランドマークに到達したら登録
            for li, lm in enumerate(landmarks):
                if cur == lm:
                    RNH[li, start] = 1

    # Local neighborhood: 各ランドマークの kNN を追加
    for li, lm in enumerate(landmarks):
        nn = indices[lm][: int(beta * k)]
        RNH[li, nn] = 1

    # 類似度を交差で計算
    similarity = RNH @ RNH.T
    M = RNH.sum(axis=1).max()
    similarity /= M

    # dissimilarity に変換
    dissimilarity = 1 - similarity
    return dissimilarity


In [ ]:
# X_prev: 下位階層のデータ, landmarks: 上位階層のランドマークインデックス
diss = compute_rnh_dissimilarity(X, landmark_ids, k=15, beta=0.2, n_walks=10, walk_length=10)
print(diss.shape)  # (n_landmarks, n_landmarks)


# generate embedding


In [ ]:
from sklearn.manifold import SpectralEmbedding

def rnh_to_probabilities(dissimilarity, sigma=1.0):
    pij = np.exp(-dissimilarity / sigma)
    pij_sym = pij + pij.T - pij * pij.T
    return pij_sym

def initialize_embedding(n_points, n_components=2, existing_coords=None, theta=0.01):
    if existing_coords is not None:
        init_coords = np.random.randn(n_points, existing_coords.shape[1]) * (1 - theta)
        init_coords[:existing_coords.shape[0], :existing_coords.shape[1]] += existing_coords * theta
        return init_coords
    return np.random.randn(n_points, n_components)

# とても遅い
def force_directed_sgd(X, pij, n_epochs=200, lr=1.0):
    n_points, n_components = X.shape
    for epoch in range(n_epochs):
        grad = np.zeros_like(X)
        for i in range(n_points):
            for j in range(n_points):
                if i == j: continue
                diff = X[i] - X[j]
                grad[i] += pij[i,j] * diff - (1 - pij[i,j]) * diff / (np.linalg.norm(diff)+1e-6)
        X -= lr * grad
    return X

# embedding = umap.UMAP(n_neighbors=15, min_dist=0.1, metric="precomputed").fit_transform(1 - pij)



In [63]:
# test
diss = compute_rnh_dissimilarity(X, landmark_ids, k=15, beta=0.2, n_walks=10, walk_length=10)
pij = rnh_to_probabilities(diss, sigma=1.0)
init_coords = initialize_embedding(len(landmark_ids), n_components=2)
# embedding = force_directed_sgd(init_coords, pij, n_epochs=200, lr=1.0)
embedding = umap.UMAP(n_neighbors=15, min_dist=0.1, metric="precomputed").fit_transform(1 - pij)
print(embedding.shape)  # (n_landmarks, 2)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/umap/umap_.py:1865: UserWarning:

using precomputed metric; inverse_transform will be unavailable



(2000, 2)


# subset associate

In [38]:
def associate_landmarks(X, landmarks, knn_graph):
    """
    非ランドマーク点をランドマークに割り当てる
    """
    n = X.shape[0]
    associate = np.full(n, -1)
    is_landmark = np.zeros(n, dtype=bool)
    is_landmark[landmarks] = True

    for u in range(n):
        # u がランドマークなら自分自身を割り当て
        if is_landmark[u]:
            associate[u] = u
            continue

        found = False
        for v in knn_graph[u]:
            if is_landmark[v]:
                associate[u] = v
                found = True
                break
            elif associate[v] != -1:
                associate[u] = associate[v]
                found = True
                break

        if not found:
            # DFSで最初に見つけたランドマークまで辿る
            stack = [u]
            visited = set()
            while stack:
                cur = stack.pop()
                if cur in visited:
                    continue
                visited.add(cur)
                if is_landmark[cur]:
                    associate[u] = cur
                    break
                stack.extend(knn_graph[cur])
    return associate


In [40]:
# test associate

from sklearn.neighbors import NearestNeighbors

# kNNグラフのインデックスリストを作成
k = 20
nbrs = NearestNeighbors(n_neighbors=k+1).fit(X)
dists, indices = nbrs.kneighbors(X)
knn_graph = [ind[1:] for ind in indices]  # 自分自身を除外

associate = associate_landmarks(X, landmark_ids, knn_graph)
print(associate.shape)  # (n_points,)


(10000,)


In [41]:
import numpy as np
from collections import defaultdict

# associate は np.array で、associate[i] = ランドマークのインデックス
def compute_landmark_groups(associate):
    groups = defaultdict(list)
    for idx, lm in enumerate(associate):
        groups[lm].append(idx)
    # ランドマークごとのサイズも
    sizes = {lm: len(indices) for lm, indices in groups.items()}
    return groups, sizes

groups, sizes = compute_landmark_groups(associate)

# 確認
for lm in groups:
    print(f"Landmark {lm}: {sizes[lm]} points -> {groups[lm][:10]}...")  # 最初の10点だけ表示


Landmark 3137: 25 points -> [0, 618, 1267, 1740, 2047, 2058, 2105, 3137, 3359, 3433]...
Landmark 1: 3 points -> [1, 3002, 4287]...
Landmark 1247: 9 points -> [2, 342, 834, 1247, 3035, 3753, 4266, 4408, 9749]...
Landmark 8767: 8 points -> [3, 1772, 2151, 3209, 5383, 7492, 7755, 8767]...
Landmark 7772: 8 points -> [4, 244, 2410, 4478, 5181, 6362, 7521, 7772]...
Landmark 4954: 5 points -> [5, 4786, 4954, 8229, 8664]...
Landmark 1082: 8 points -> [6, 558, 1082, 1497, 2595, 4175, 5562, 7680]...
Landmark 6811: 10 points -> [7, 450, 2004, 2448, 4737, 4758, 4827, 6811, 7098, 8839]...
Landmark 9827: 4 points -> [8, 3701, 6992, 9827]...
Landmark 5791: 2 points -> [9, 5791]...
Landmark 4788: 7 points -> [10, 1370, 2473, 2520, 4788, 6159, 9975]...
Landmark 9313: 6 points -> [11, 2104, 5215, 6597, 7082, 9313]...
Landmark 2700: 5 points -> [12, 800, 2700, 5165, 8243]...
Landmark 2243: 4 points -> [13, 2243, 3429, 8536]...
Landmark 6534: 6 points -> [14, 1716, 5280, 6400, 6534, 7518]...
Landmark 8076

In [59]:
# sisesのヒストグラム
import plotly.express as px
fig = px.histogram(x=list(sizes.values()), nbins=50, title="Landmark Group Sizes")
fig.show()


In [60]:
# landmarkをそれぞれランダムな座標に表示し、plotlyで可視化. ランドマークに属する点のサイズを不透明な円で
import plotly.graph_objects as go
import numpy as np
import random
from sklearn.decomposition import PCA
import umap
# landmark_coords = PCA(n_components=2).fit_transform(X[landmark_ids])
landmark_coords = umap.UMAP(n_components=2).fit_transform(X[landmark_ids])
point_coords = np.zeros((X.shape[0], 2))
point_sizes = np.zeros(X.shape[0])  
for lm_idx, lm in enumerate(landmark_ids):
    pts = groups[lm]
    for pt in pts:
        point_coords[pt] = landmark_coords[lm_idx]
        point_sizes[pt] = 5 + sizes[lm] * 0.1  # サイズはランドマークに属する点の数に比例

fig = go.Figure()

fig.add_trace(go.Scatter(x=point_coords[:,0], y=point_coords[:,1],
                         mode='markers',
                         marker=dict(size=point_sizes, color='skyblue', opacity=0.2,
                                     line=dict(width=0.5, color='DarkSlateGrey')),
                         name='Points'))
fig.add_trace(go.Scatter(x=landmark_coords[:,0], y=landmark_coords[:,1],
                         mode='markers',
                         marker=dict(size=1, color='black'),
                         name='Landmarks'))
fig.update_layout(title='Landmarks and Associated Points',
                  xaxis_title='X',
                  yaxis_title='Y')
fig.show()

In [64]:
# embedding
fig = go.Figure()

fig.add_trace(go.Scatter(x=embedding[:,0], y=embedding[:,1],
                         mode='markers',
                         marker=dict(size=point_sizes, color='skyblue', opacity=0.2,
                                     line=dict(width=0.5, color='DarkSlateGrey')),
                         name='Points'))
fig.add_trace(go.Scatter(x=embedding[:,0], y=embedding[:,1],
                         mode='markers',
                         marker=dict(size=1, color='black'),
                         name='Landmarks'))
fig.update_layout(title='Landmarks and Associated Points',
                  xaxis_title='X',
                  yaxis_title='Y')
fig.show()

In [61]:
# landmarkのみ
fig = px.scatter(x=landmark_coords[:,0], y=landmark_coords[:,1],
                 title='Landmarks Only',
                 labels={'x': 'X', 'y': 'Y'})
fig.update_traces(marker=dict(size=2, color='black'))
fig.show()

In [62]:
# pca

pca_landmark_coords = PCA(n_components=2).fit_transform(X[landmark_ids])
fig = px.scatter(x=pca_landmark_coords[:,0], y=pca_landmark_coords[:,1],
                 title='Landmarks PCA',
                 labels={'x': 'PC1', 'y': 'PC2'})
fig.update_traces(marker=dict(size=2, color='black'))
fig.show()

In [52]:
fig = go.Figure()

size_attr = np.array([sizes[lm] for lm in landmark_ids])
fig.add_trace(go.Scatter(x=landmark_coords[:,0], y=landmark_coords[:,1],
                            mode='markers',
                            marker=dict(size=size_attr, color='skyblue', opacity=0.3, line=dict(width=1, color='DarkSlateGrey')),
                           
                            name='Landmarks'))
fig.add_trace(go.Scatter(x=landmark_coords[:,0], y=landmark_coords[:,1],
                            mode='markers',
                            marker=dict(size=2, color='black'),
                            name='Landmarks'))
fig.show()



In [ ]:
"""
X = original_data

for level in hierarchy_levels:
    indices, P = compute_probabilities(X, k=level.k)
    landmark_ids, visits = select_landmarks(X, num_landmarks=level.num_landmarks, k=level.k)
    
    DRNH = compute_dissimilarity(X, landmark_ids, beta=level.beta)
    embedding = generate_embedding(DRNH)
    
    associations = associate_landmarks(X, landmark_ids)
    X = X[landmark_ids]  # 次の階層の入力はランドマーク

"""